<a href="https://colab.research.google.com/github/dajub/simple-rhythm-maker/blob/master/simple_rhythm_makerl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf


TensorFlow 2.x selected.


In [ ]:
# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dense
# from keras.layers import Dropout
# from keras.callbacks import ModelCheckpoint
# from keras.utils import np_utils

In [ ]:
# import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import io
%matplotlib inline

In [ ]:
!pip install --upgrade --quiet gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
# %cd /content/drive/My Drive
# !ls

In [ ]:
# Get the file with the transcriptions
name = '/content/drive/My Drive/transcriptions.gsheet'
name = 'transcriptions'
worksheet = gc.open(name).sheet1
rows = worksheet.get_all_values()
print(rows)
trans = pd.DataFrame(rows[1:])
trans.head()

[['1', 'e', '&', 'a', '2', 'e', '&', 'a', '3', 'e', '&', 'a', '4', 'e', '&', 'a', '1', 'e', '&', 'a', '2', 'e', '&', 'a', '3', 'e', '&', 'a', '4', 'e', '&', 'a'], ['hb', '', 'h', '', 'hs', '', 'h', '', 'hb', '', 'h', '', 'hs', '', 'h', '', 'hb', '', 'h', '', 'hs', '', 'h', '', 'hb', '', 'h', '', 'hs', '', 'h', ''], ['hb', '', 'h', '', 'hsb', '', 'h', '', 'hb', '', 'h', '', 'hsb', '', 'h', '', 'hb', '', 'h', '', 'hsb', '', 'h', '', 'hb', '', 'h', '', 'hsb', '', 'h', ''], ['hb', '', 'h', '', 'hs', '', 'h', '', 'hb', '', 'hb', '', 'hs', '', 'h', '', 'hb', '', 'h', '', 'hs', '', 'h', '', 'hb', '', 'hb', '', 'hs', '', 'h', ''], ['hb', '', 'hb', '', 'hs', '', 'h', '', 'hb', '', 'hb', '', 'hs', '', 'h', '', 'hb', '', 'hb', '', 'hs', '', 'h', '', 'hb', '', 'hb', '', 'hs', '', 'h', ''], ['hb', '', 'hb', '', 'hs', '', 'h', '', 'h', '', 'h', '', 'hs', '', 'hb', '', 'hb', '', 'hb', '', 'hs', '', 'h', '', 'h', '', 'h', '', 'hs', '', 'hb', ''], ['hb', '', 'h', 'b', 'hs', '', 'h', '', 'hb', '', 'h', 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,hb,,h,,hs,,h,,hb,,h,,hs,,h,,hb,,h,,hs,,h,,hb,,h,,hs,,h,
1,hb,,h,,hsb,,h,,hb,,h,,hsb,,h,,hb,,h,,hsb,,h,,hb,,h,,hsb,,h,
2,hb,,h,,hs,,h,,hb,,hb,,hs,,h,,hb,,h,,hs,,h,,hb,,hb,,hs,,h,
3,hb,,hb,,hs,,h,,hb,,hb,,hs,,h,,hb,,hb,,hs,,h,,hb,,hb,,hs,,h,
4,hb,,hb,,hs,,h,,h,,h,,hs,,hb,,hb,,hb,,hs,,h,,h,,h,,hs,,hb,


In [ ]:
# ideally put this into another file and just call it, but in meantime...
number_of_instruments = 3
# dictionary for converting from letters to binary tuple notes
data = {}
# for each groove...
for j in range(len(trans)):
    # make a new dictionary from groove# to a list of notes
    data[j] = []
    # for each lettered note . . .
    for i in range(len(trans.columns)):
        #  make a list of 0's [meaning no instruments]
        note = [0]*number_of_instruments
        # print(trans.iloc[j,i])
        # and for each letter, turn the corresponding instrument on
        human = str(trans.iloc[j,i])
        if "h" in human:
            note[0] = 1
        if "b" in human:
            note[1] = 1
        if "s" in human:
            note[2] = 1
        # and append it to that groove's list as a tuple
        data[j].append(tuple(note))
# print(data)



In [ ]:
# convert that binary notation into a df
list_version =[]
for key in data.keys():
    list_version.append(data[key])
binary = pd.DataFrame(list_version)
print(binary.shape)
binary.head()

In [ ]:
# for computing the distance between two tuples in n-dimensional space
def sq_distance(a, b):
  distance = 0 
  for i in range(len(a)):
    # print(i)
    distance += (a[i] - b[i])**2
    # print(a[i],b[i],distance)
  return distance


In [ ]:
# The distance between two sequences of notes
def list_sq_distance(a,b):
  distance = 0
  assert(len(a) == len(b))
  for i in range(len(a)):
    distance += sq_distance(a[i], b[i])
  return distance


In [ ]:
# Make a dictonary for all of the unique notes that are played in the data
# Dict from those tuples to integers and the reverse

# Data all in one array
binary_wrapped = np.array(binary).ravel()

uniqueNotes, uniqueNotesCounts = np.unique(binary_wrapped, return_counts=True )
# a dictionary of each note to a corresponding integer
noteToInt = dict(zip(uniqueNotes, list(range(0, len(uniqueNotes)))))
# a dictionary of each integer to a note
intToNote  = {i: n for n, i in noteToInt.items()}

# how many unique notes do we have
print("number of notes: " + str(uniqueNotes.shape[0]))
# the counts of each note in the larger thing
print(uniqueNotesCounts)
# the mapping to integers
print(noteToInt)
# the inverse of that dictionary 
print(intToNote)

print(binary.shape)
print(binary[0].shape)


In [ ]:
# Collect the Training Data
# Define number of beats in training sequences
sequenceLength = 16
rhythmLength = binary.shape[1]
numberOfRhythms = len(binary)
numberOfRythmsToUse =  int(numberOfRhythms * 1) - 1
print(str(numberOfRythmsToUse) + ' / ' + str(numberOfRhythms) + " rhythms used to train.")

# Define empty arrays for train data
trainNotes= []
targetNotes = []



for i in range(numberOfRythmsToUse):
  # beatsInRythm = int(len(binary[0])/4)
  doubleBeat = list(binary.iloc[i,:]) + list(binary.iloc[i,:]) # double size of beat so it wraps around
  # print((doubleBeat))
  for j in range(0,rhythmLength):
    addThis = doubleBeat[j:j+sequenceLength]
    # print(addThis)
    # print((addThis))
    trainNotes.append(addThis)
    # targetNotes.append(doubleBeat[j+1:j+1+sequenceLength])
    targetNotes.append(doubleBeat[j+sequenceLength])

print("Number of patterns: " + str(len(trainNotes)) + "==" + str(len(targetNotes)))
print(trainNotes)
    
  
 

  


In [ ]:
n_patterns = len(trainNotes)
# Turn this from binary form into integer form using the dictionary we made above:
intTrainNotes = trainNotes.copy()
intTargetNotes = targetNotes.copy()
print(intTargetNotes)
print(intTrainNotes)
intTrainNotes = [[noteToInt[n] for n in r] for r in intTrainNotes]
# intTargetNotes = [[noteToInt[n] for n in r] for r in intTargetNotes]
intTargetNotes = [noteToInt[n] for n in intTargetNotes]
print(intTrainNotes[0], trainNotes[0])
print(noteToInt)

In [ ]:
# make this into one-hot encoded rythms where we have each note is an array of 
# length of len(noteToInt) and int it corresponds to is made a 1
X = np.reshape(intTrainNotes, (n_patterns, sequenceLength, 1))
y = tf.keras.utils.to_categorical(intTargetNotes)
# print(trainNotes[0])
# print(intTrainNotes[0])
# print(oneHotTrainNotes[0])
# print(oneHotTargetNotes[0])
print(X[0])
print(y[0])

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(tf.keras.layers.Dropout(.2))
# model.add(tf.keras.layers.Lambda(lambda x: tf.expand_dims(model.output, axis=-1)))
model.add(tf.keras.layers.LSTM(256))
model.add(tf.keras.layers.Dropout(.02))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam')


In [ ]:
print(X)
print(y)

In [ ]:
history = model.fit(X,y, epochs=160)

In [ ]:
print(intToNote)

{0: (0, 0, 0), 1: (0, 0, 1), 2: (0, 1, 0), 3: (1, 0, 0), 4: (1, 0, 1), 5: (1, 1, 0), 6: (1, 1, 1)}


In [ ]:
# make a seed for prediction:
# seed = np.array(intTrainNotes[0])
seed1 = np.array([5,3,3,0,4,1,3,0,5,3,3,3,4,3,3,1])
seed2 = np.array([5,0,3,0,4,0,3,0,5,0,3,0,4,0,3,0])
seed3 = np.array([5,0,3,1,4,0,5,0,5,1,3,1,4,0,3,3])
seed4 = np.array([3,3,3,1,4,3,5,0,5,3,3,0,4,3,3,1])
seed5 = np.array([1,2,3,4,5,6,0,1,2,3,4,5,6,0,1,1])

seeds = [seed1, seed2, seed3,seed4, seed5]



predictedNotes = []
results = []
# print(seed)
numberOfNotesToPredict = 64
for j in range(len(seeds)):
  predictedNotes = []
  seed = seeds[j]
  for i in range(numberOfNotesToPredict):
    # print(seed)
    x = np.reshape(seed, (1,len(seed),1))
    prediction = model.predict(x, verbose=0)
    # print(prediction)
    note = np.argmax(prediction)
    predictedNotes.append(note)
    # print(note)
    # seed.add(note)
    temp = seed.tolist()
    temp.append(note)
    seed = np.array(temp[1:len(temp)])
    # seed.append(note)
    # seed = seed[1:len(seed)]
  results.append(predictedNotes)
    

print(predictedNotes)
# for n in predictedNotes:
#   print(intToNote[n])



  


In [ ]:
for i in range(len(results)):
  print(results[i])

In [ ]:
############################################################################################################################################
# Here we use a simple probabalistic approach where each quantified note is just randomly chosen from the input. 

In [ ]:
binary.head()

In [ ]:
# Sum up all of the options in one column, i.e. position in the beat
prob = {}
numberOfSubdivisions = 32
for i in range(numberOfSubdivisions):
  p = []
  for n in binary.iloc[:,i]:
    p.append(noteToInt[n])
  prob[i] = p


In [ ]:
print(len(prob))

32


In [ ]:
beats = []
numberOfBeats = 3
for i in range(numberOfBeats):
  beat = []
  for j in range(64):
  # print(prob[i])
    b = np.random.choice(prob[j%numberOfSubdivisions])
    beat.append(b)
  beats.append(beat)
# print(beats)
# print(intToNote)
for i in range(numberOfBeats):
  print(beats[i])

In [ ]:
# See how many of these are not in binary
binary2 = [[intToNote[n] for n in r] for r in beats]
print(binary2)

In [ ]:
bn = np.array(binary).tolist()
lame = bn[0]
print(lame)
binary2.append(lame)

In [ ]:
uniqueBeats = []
for beat in binary2:
  # print(beat)
  if beat not in bn:
    uniqueBeats.append(beat)

In [ ]:
print("Created " + str(len(binary2)) + " beats and have " + str(len(uniqueBeats)) + " unique")

Created 4 beats and have 3 unique


In [ ]:
df = pd.DataFrame.from_records(binary2)
df.head()

In [ ]:
# print(binary.iloc[38,:].tolist())
b = binary.iloc[31,:].tolist()
a = [noteToInt[n] for n in b]
e = [noteToInt[n] for n in binary.iloc[12,:].tolist()]
f = [noteToInt[n] for n in binary.iloc[36,:].tolist()]

print(a)
print(e)
print(f)

In [ ]:
rando = []
for i in range(64):
  rando.append(np.random.randint(7))
print(rando)